<center>
<img src='Logo.jpg' height=30% width=30%>
</center>

# **项目实操：视频转字符画**

## **心生向往**

看到有沙雕网友将图片和视频做成了（动态）字符画，看上去非常地酷炫！于是我们便想自己写写代码实现这个功能。

通过这个（微型）项目我们可以初步体验：  
> 如何查找学习资源  
> 如何阅读帮助手册  
> 如何规划程序设计  
> 如何使用脚本式文本编辑器  

## **大卸八块**

首先我们想一想，如何将图片转化为字符画？

> 图片由一颗颗像素组成。而在字符画中，一个字符其实就对应了一个小小的像素区块，远远望去，这个区块有多“白”，取决于这个区块有多少像素是“亮”的！

> 于是，我们便可将图片裁剪成一个一个小块，评估这个小块的“亮度”，然后选择对应的字符来代替它。

接着对这个功能进行模块化的分析。

> 需求输入：一个视频文件  
> 需求输出：动态的字符画，用命令行即可

当中的步骤大致可分为：  
> 1. 读取文件  
> 2. 将视频分帧成画  
> 3. 将画变为灰白色  
> 4. 裁剪成块，评估每块亮度  
> 5. 用字符代替  
> 6. 按帧输出  

输出时有两个方案：  
> - 一边计算一边输出。占用内存少，然而如果计算量大或者电脑速度跟不上，将会影响成果的流畅度。      
> - 将所有数据计算完统一输出。占用内存大，然而可以保证流畅。  

我们采用第二种方法。

然后我们就可以一步一步的实现它了。

---

## **目无全牛**

我们心无旁骛地专注于一项功能，逐个击破。

> Python标准库中用于读取文件的函数为`open()`。但由于Python读取文件为一个**对象**，而这个对象的方法由对其操作的库定义，所以一般的库都会包含重写过的读取方法。所以我们可以先跳过第一步。

**分帧**

百度一下视频分帧的方法，就可以知道用的是一个叫[opencv](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html)的第三方库。

> 首先打开命令行，输入`pip install opencv-python`安装这个库。

安装完成之后就可以导入这个库了，这里有点特殊用`cv2`名称导入这个库：

In [ ]:
import cv2

查阅教程**Gui Features on OpenCV > Getting Started with Videos > Playing Video from file**，使用`VideoCapture()`方法加载文件：

In [ ]:
video = cv2.VideoCapture("/Users/vector/Documents/test.mkv")

这里的文件路径需要自己修改一下，我使用的macOS中文件路径是这么写的。

加载之后就可以用read()方法读取了。我们先用`help()`函数查看一下read是怎么使用的：

In [ ]:
help(video.read)

可以看到read()方法读取了一帧图像，返回两个数据：

In [ ]:
retval ,image = video.read()

In [ ]:
retval

这是用于判断是否有下一帧的布尔值。

In [ ]:
image

这里就是一帧图像的数据了。可以看到这里有三层列表的嵌套。最内层的一层列表，如[6, 10, 5]，存储了一个像素点的RBG值。以像素点为单位的二维数组即一幅图像。。

> 至此我们明白，运行一次read()即读取一帧图像。视频读取和分帧的功能就这样实现了。

**转换为灰度图**

尽管将RGB图像转化为灰度图有算法可以查询，但是我们还是想偷个懒。看到opencv手册中**Image Processing in OpenCV > Changing Colorspaces**，我们有现成的函数可以使用：

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
gray

现在每个值代表一个像素点的`灰度`。

**裁剪成块**

由于一个字符比一个像素点大很多，所以我们通过决定像素的个数来确定块数。

In [ ]:
show_heigth = 30
show_width  = 80

继续使用别人的轮子：手册中**Image Processing in OpenCV > Geometric Transformations of Images** 中为我们提供了现成的分块方法`cv2.resize()`。

In [ ]:
help(cv2.resize)

In [ ]:
gray_2 = cv2.resize(gray,(show_width, show_heigth))
gray_2

每块的亮度就用该块中所有像素灰度的平均值表示。

**替换对应的字符**

已经有人为我们整理好了一串字符由黑到白的排序，这里我们直接引用：

In [ ]:
ascii_char = "$@B%8&WM#*oahkbdpqwmZO0QLCJUYXzcvunxrjft/\|()1{}[]?-_+~<>i!lI;:,\"^`'. "

In [ ]:
char_len = len(ascii_char) #获取这串字符的长度

In [ ]:
char_len

灰度的一共有256个值（0-255的整数值）。我们将这256个值大约等分分为70档，每一档对应一个字符。比如说一个字符的灰度为120，就是这么对应：

In [ ]:
pixel = 120
ascii_char[int(pixel / 256 * char_len)] #int 用于取整

用for循环对裁剪好的所有块全都执行操作：

In [ ]:
text ='' 
for pixel_line in gray_2:
    for pixel in pixel_line:
        text += ascii_char[int(pixel / 256 * char_len)]
    text += '\n' #换行

In [ ]:
print(text)

如此我们便完成了对一帧图像的处理。接下来只要用循环重复上面的过程，并将所有得到的结果存储到一个列表里，到时候挨个输出就行了。

**按帧输出**

依次输出时，由于所有的数据已经运算完毕，所以输出的过程会非常快，体现在视频会放的特别快。为了让视频保持原速，我们需要让Python在每输出完一帧图像后等待一段时间。这时候我们就需要用标准库`time`中的`sleep()`函数。可以先体会一下这个函数的作用：

In [ ]:
import time

In [ ]:
help(time.sleep)

In [ ]:
time.sleep(2)
print('delayed 2 seconds')

## **合而为一**

我们把上面的思路稍微整合一下，套上一个while循环，就可以获得一个具有完整功能、可以独立出来的代码块。

```python
import time

import cv2

show_heigth =   30            
show_width = 80

ascii_char = "$@B%8&WM#*oahkbdpqwmZO0QLCJUYXzcvunxrjft/\|()1{}[]?-_+~<>i!lI;:,\"^`'. "
#生成一个ascii字符列表

char_len = len(ascii_char)#获取字符个数

vc = cv2.VideoCapture("/Users/vector/Documents/test2.mp4")          #加载视频

if vc.isOpened():   #判断是否正常加载，如果正常加载就读取第一帧
    rval , frame = vc.read()
else:
    rval = False
    
frame_count = 0 #已经运算完的帧数

outputList = [] #初始化输出列表

while rval:   #如果下一帧存在，就进入循环  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  #转化成灰度图
    gray = cv2.resize(gray,(show_width,show_heigth))#切割灰度图
    
    text = ""
    for pixel_line in gray:
        for pixel in pixel_line:                    
            text += ascii_char[int(pixel / 256 * char_len )]
        text += "\n"         #完成一帧的运算

    outputList.append(text) #将结果存储的列表中
    
    frame_count = frame_count + 1   #已经运算完的帧数+1  
    
    if frame_count % 100 == 0:
        print("已处理" + str(frame_count) + "帧") #每运算完100帧汇报一次，进度检测
        
    rval, frame = vc.read() #读取下一帧
    
print("处理完毕")

interval = 1.0 / 30 #确定输出间隔

for frame in outputList:       
    print(frame) #输出一帧
    print()
    print()
    
    time.sleep(interval) #等待一个间隔
```

虽然这段代码在交互式文本编辑器中是可执行的，但是效果并不太理想。我们想用脚本式文本编辑器来实现这段代码。

<center>
<img src='Python.jpg' height=90% width=90%>
</center>